In [4]:
import numpy as np
import re
from nltk.tokenize import sent_tokenize, word_tokenize

import spacy
import scispacy


In [6]:
import xml.etree.ElementTree as ET
import random
import os
import numpy as np



TRAINNING_DATA_DIR = "./corpus/i2b2/2012-07-15.original-annotation.release/"
TEST_DATA_DIR = "./corpus/i2b2/ground_truth/merged_xml/"
SAVE_DIR = "./corpus/i2b2/"


import transformers
from transformers.tokenization_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)


nlp = spacy.load("en_core_sci_sm")



def file_name(file_dir):
    L=[]
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.xml':
                L.append(file)
    return L



def create_entry(tlink, text, f = ''):
    id = f[:-4] +"_"+ str(tlink.attrib['id'] )
    target = tlink.attrib['fromText'] + " " + tlink.attrib['toText']
    label = tlink.attrib['type'].upper()
    if label == '':
        return ""
    
    text = text.replace("\n", " ")
    
    #print(id + "\t"+target + "\t" + label)
    return id + "\t" + target + "\t" + text + "\t"+ label


def data_process(inDIR, outFile):
    fileList = file_name(inDIR)
    lableType = set()
    outFile = open(outFile, "w+")
    
    fileList.sort()
    
    feature_count = 0
    count = 0
    duplicates_count = 0
    
    final = []
    
    for f in fileList:
        
        #print(f, end=' ')
        linkNO = 0
        inFile = open(inDIR + f, "r")
        xmlString = ""
        for lines in inFile.readlines():
            xmlString += lines.replace(" & ", " ").replace("&", " and ")
        inFile.close()

        parser = ET.XMLParser(encoding="utf-8")
        root = ET.fromstring(xmlString, parser=parser)
        text = root.find("TEXT").text         
        tags = root.find("TAGS")
    
        
        text = text.split("\n")[5:]
        text = "\n".join(text)
        
        text = re.sub(r"\n\d+\. ","\n* ",text)
        text = re.sub(r"\n\d+\) ","\n* ",text)
        text = re.sub(r"\n# ","\n* ",text)
        text = re.sub(r"\n-(-*)","\n* ",text)
        text = re.sub(r"\n\* ","\n· ",text)
        
        doc = nlp(text)
        sentences = [str(i) for i in list(doc.sents)]
        
    
        for tlink in tags.findall("TLINK"):
            
                id = f[:-4] +"_"+ str(tlink.attrib['id'] )
                to_text = tlink.attrib['fromText']
                from_text = tlink.attrib['toText']
                
                
                ss = [s for s in sentences if from_text in s and to_text in s]
                if len(ss) > 1:
                    duplicates_count += len(ss)
                    print(to_text)
                    print(from_text)
                    print(ss)
                    print(f)
                
                #If not a duplicate,
                #append to list of features that are being saved
                else:
                    for sentence in sentences:
                        if to_text in sentence and from_text in sentence:
                            #print(id + "\t"+to_text + "\t" + from_text + "\t[" + sentence + "]")
                            final.append(create_entry(tlink, sentence, f))
                            feature_count += 1


                     
    final2 = [i + "\n" for i in final if i != ""]
    print(final2[:20])  
    outFile.writelines(final2)

    print("Duplicate count: " + str(duplicates_count))
    print("Link count: " + str(feature_count))
    
    print("*"*80)
    outFile.close()
                
data_process( TRAINNING_DATA_DIR , SAVE_DIR + "remove_duplicate_features_train.txt")
data_process( TEST_DATA_DIR , SAVE_DIR + "remove_duplicate_features_test.txt")
print("Done")


HIV positive
HIV positive
['HISTORY OF PRESENT ILLNESS :\nThe patient is a 28-year-old woman who is HIV positive for two years .\n', 'She claims she does not know why she is HIV positive .\n']
1.xml
nausea
left upper quadrant pain
['She presented with left upper quadrant pain as well as nausea and vomiting which is a long-standing complaint .\n', 'She has complaints of nausea and vomiting as well as left upper quadrant pain on and off getting progressively worse over the past month .\n']
1.xml
vomiting
left upper quadrant pain
['She presented with left upper quadrant pain as well as nausea and vomiting which is a long-standing complaint .\n', 'She has complaints of nausea and vomiting as well as left upper quadrant pain on and off getting progressively worse over the past month .\n']
1.xml
nausea
vomiting
['She presented with left upper quadrant pain as well as nausea and vomiting which is a long-standing complaint .\n', 'She has complaints of nausea and vomiting as well as left upper 

bands
lymphs
['On 9/5/93 , hematocrit 48 , white blood count 11.2 , neutrophiles 67 , bands 14 , lymphs 11 , monos 6% , meta 2% , platelets 220,000 .\n', 'Differential was polys 59 , bands 3 , lymphs 25 .\n']
121.xml
chloramphenicol
ceftriaxone
['IV ampicillin , chloramphenicol , ceftriaxone , and culture were sent .\n', 'Infectious Disease :\nAmpicillin and chloramphenicol were given at Noughwell Entanbon Health , and ceftriaxone was given at Fairm of Ijordcompmac Hospital .\n']
121.xml
dopa
dopa
['A second intravenous was started and given normal saline bolus , change in intravenous fluids to D5 water with 80 mEq of bicarbonate , plus 40 mEq of KCL at 45 cc and apos;s per hour , bicarbonate given 2 mEq per kilo , attempted to start dopa for poor perfusion but worsening perfusion .\n', 'Therefore , dopa was stopped .\n']
121.xml
sodium bicarbonate
bolus
['At 9/7/93 at 5:00 a.m. , sodium bicarbonate given 60 mEq , calcium bolus 10 cc and apos;s given .\n', 'Total volume given as bolus 

admission
admission
['HISTORY OF PRESENT ILLNESS :\nThis 82 year old retired engineer has had multiple admissions to the Ph University Of Medical Center for multiple strokes and a spontaneous subdural hematoma and prostatic hypertrophy .\n', 'He has been managing at home on restricted activity but able to get around with a walker but on the day before admission he became increasing dyspneic and on the day of admission he collapsed and was unable to walk to his bed .\n']
126.xml
the time
discharge
['At the time of discharge , it was 1.9 , for which she received 4 mg.\n', 'Her wound remained benign and was healing nicely at the time of discharge .\n']
141.xml
discharge
time
['At the time of discharge , it was 1.9 , for which she received 4 mg.\n', 'This was postoperative day # 6 , at which time she is ready for discharge .\n', 'Her wound remained benign and was healing nicely at the time of discharge .\n']
141.xml
postoperative day # 1
postoperative day # 1
['On postoperative day # 1 , t

rehabilitation
the rehabilitation center
['He was transferred to Erich Lindemann Mental Health Center from the rehabilitation center where he had been discharged to on 2010-08-25 following his CABG/MVR .\n', 'At the rehabilitation center the patient developed shortness of breath .\n']
152.xml
the rehabilitation center
the rehabilitation center
['He was transferred to Erich Lindemann Mental Health Center from the rehabilitation center where he had been discharged to on 2010-08-25 following his CABG/MVR .\n', 'At the rehabilitation center the patient developed shortness of breath .\n']
152.xml
rehabilitation
transferred
['He was transferred to Erich Lindemann Mental Health Center from the rehabilitation center where he had been discharged to on 2010-08-25 following his CABG/MVR .\n', "On 09-02 it was decided that the patient was stable and ready to be transferred back to rehabilitation , however case management was unable to get authorization from the patient 's insurance company and he 

delivery
delivery
['Unremarkable pregnancy except for minor fullness of the left renal pelvis reported during the week prior to delivery .\n', 'Baby was delivered by repeat C-section with rupture of membranes at delivery .\n']
162.xml
admitted
admitted
['The patient had KUB that showed multiple dilated loops of small bowel and was admitted for a partial small bowel obstruction .\n', 'An nasogastric tube was placed and had an H2 blocker started , and was admitted for bowel rest and decompression .\n']
166.xml
CMO
CMO
['After a long discussion , patient and family asked to be made CMO .\n', 'Davison died approximately 24 hours after decision to be CMO , on 2018-05-01 .\n']
172.xml
The mass
The mass
['The mass grew larger , to the size of a baseball .\n', 'The mass appears to be cystic in nature and has been aspirated at least 3 times , which decrease in size each time .\n']
172.xml
here
transferred
['Pt was transferred to New England Baptist Hospital , where his surgeon is located .\n', 

...

intubated
intubated
['The patient tolerated the procedure well and was transferred to the unit , intubated on Milrinone , Levophed and Propofol drips .\n', 'Pulmonary :\nThe patient remained intubated until postoperative day number five .\n', 'Pulmonary :\nThe patient remained intubated until postoperative day number five .\n']
78.xml
intubated
postoperative day number five
['Pulmonary :\nThe patient remained intubated until postoperative day number five .\n', 'Pulmonary :\nThe patient remained intubated until postoperative day number five .\n']
78.xml
Milrinone
Milrinone
['The patient tolerated the procedure well and was transferred to the unit , intubated on Milrinone , Levophed and Propofol drips .\n', 'Milrinone was continued until postoperative day number three , at which point he was having good enough cardiac output that he did not need these drips to help maintain his blood pressure anymore .\n']
78.xml
the floor
the floor
['He was sent to the floor from the cardiac surgery rec

Levofloxacin
Levofloxacin
['She was started on Levofloxacin but the patient became hypotensive at that point with blood pressure of 70/45 and received a normal saline bolus to boost her blood pressure to 99/60 ; however the patient was admitted to the Medical Intensive Care Unit for overnight observation because of her somnolence and hypotension .\n', 'HOSPITAL COURSE :\nAfter transfer to the Floor , the patient was continued on Levofloxacin for a ten to 14 day course for presumptive pneumonia .\n']
792.xml
services
services
['Physical Therapy felt that the patient was cleared for transfer home with services .\n', "Occupational Therapy was also in agreement with the patient 's disposition home with services .\n"]
792.xml
services
home
['Physical Therapy felt that the patient was cleared for transfer home with services .\n', "Occupational Therapy was also in agreement with the patient 's disposition home with services .\n"]
792.xml
CT
CT
['Service :\nGGI\nOPERATIONS AND PROCEDURES :\nOT

HD
HD
['Service :\nMEDICINE\nHistory of Present Illness :\n69 y/o female wtih PMH significant for ESRD on HD , type 2 DM , and recent PE resulting in PEA arrest admitted through the Juan with sepsis of unknown etiology .\n', 'Pt was recently admitted to The Hospital for Orthopedics from 12-07 thorugh 12-14 with hypotension thought to be secondary to overdiuresis at HD.\n', 'Brief Hospital Course :\nA/P :\n69 y/o female wtih PMH significant for ESRD on HD , type 2 DM , and recent PE resulting in PEA arrest admitted through the Juan with sepsis of unknown etiology .\n', 'Cultures drawn off the HD cath are negative to date .\n', '· ESRD on HD - Pt has ESRD secondary to her DM and is on HD .\n']
812.xml
recent PE
recent PE
['Service :\nMEDICINE\nHistory of Present Illness :\n69 y/o female wtih PMH significant for ESRD on HD , type 2 DM , and recent PE resulting in PEA arrest admitted through the Juan with sepsis of unknown etiology .\n', 'Brief Hospital Course :\nA/P :\n69 y/o female wtih 

two weeks prior
admission
['The patient reported feeling at his baseline with chronic mild dyspnea on exertion until two weeks prior to admission .\n', 'Over the two weeks prior to admission he had noticed an increasing heart rate from his baseline in the 60s to a heart rate in the 120s .\n']
817.xml
admission
admission
['He was seen by his primary care physician the day before admission and complained of two weeks of increasing shortness of breath and fatigue , weight gain and abdominal fullness .\n', 'The patient reported feeling at his baseline with chronic mild dyspnea on exertion until two weeks prior to admission .\n', 'Over the two weeks prior to admission he had noticed an increasing heart rate from his baseline in the 60s to a heart rate in the 120s .\n', 'Upon admission it was unclear whether or not the patient was volume overloaded or volume depleted .\n', "The patient 's Digoxin had been weaned off prior to his admission .\n", "· Congestive heart failure - During the initia

Klonopin
Inderal
['Per report , there were empty bottles of Inderal and Klonopin found at bedside , as well as 02-26 full bottles of geodon , gabapentin , Lescol and paroxitine .\n', '48yo man with h/o HCV , bipolar DO , h/o suicide attempts , a/w overdose of Inderal , Klonopin , Geodon , s/T Jackson stay with intubation for airway protection , with question of L retrocardiac infiltrate , now doing well .\n']
88.xml
Klonopin
Klonopin
['Per report , there were empty bottles of Inderal and Klonopin found at bedside , as well as 02-26 full bottles of geodon , gabapentin , Lescol and paroxitine .\n', '48yo man with h/o HCV , bipolar DO , h/o suicide attempts , a/w overdose of Inderal , Klonopin , Geodon , s/T Jackson stay with intubation for airway protection , with question of L retrocardiac infiltrate , now doing well .\n']
88.xml
Inderal
Inderal
['Per report , there were empty bottles of Inderal and Klonopin found at bedside , as well as 02-26 full bottles of geodon , gabapentin , Lesco